# Summary

----

# Imports

In [1]:
%run imports.ipynb

2016-08-27 20:48:35.703841


In [2]:
%run db.ipynb

MySQL database already running...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-27 20:48:35.892527


In [3]:
NOTEBOOK_NAME = 'rosetta_ddg'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Load data

In [4]:
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

In [5]:
!7za x -aos -bb0 -bd -o"$NOTEBOOK_NAME" ../downloads/rosetta_ddg.zip


7-Zip (a) [64] 15.09 beta : Copyright (c) 1999-2015 Igor Pavlov : 2015-10-16
p7zip Version 15.09 beta (locale=en_CA.UTF-8,Utf16=on,HugeFiles=on,64 bits,64 CPUs x64)

Scanning the drive for archives:
1 file, 14095905 bytes (14 MiB)

Extracting archive: ../downloads/rosetta_ddg.zip
--
Path = ../downloads/rosetta_ddg.zip
Type = zip
Physical Size = 14095905
Comment = 37d405af2dac41477c689e6e63d5f5c2b9f5a665

Everything is Ok

Folders: 19
Files: 0
Size:       0
Compressed: 14095905


In [6]:
ls $NOTEBOOK_NAME

alascan_gpk.tsv.bz2      guerois.tsv.bz2         potapov.tsv.bz2
citations.tsv.bz2        kellogg.tsv.bz2         references.tsv.bz2
curatedprotherm.tsv.bz2  kortemme_baker.tsv.bz2
ddg-master/              pdbs.tsv.bz2


In [7]:
ls $NOTEBOOK_NAME/ddg-master/input/

csv/  json/  pdbs/  README.rst  tools/


In [8]:
ls $NOTEBOOK_NAME/ddg-master/input/csv/

alascan-gpk.csv      guerois.csv  kortemme_baker.csv  potapov.csv
curatedprotherm.csv  kellogg.csv  pdbs.csv            references.csv


In [9]:
ls $NOTEBOOK_NAME/ddg-master/input/json/

alascan-gpk.json      guerois.json  pdbs.json     references.json
curatedprotherm.json  kellogg.json  potapov.json


# Load data

In [10]:
column_names = [
    'RecordID', 'PDBFileID', 'Mutations', 'DDG', 'ResidueExposures', 
    'DSSPTypes', 'DSSPSimpleTypes', 'IndividualDDGs'
]

def load_rosetta_ddg(filename):
    
    def _parse_mutations(x):
        pdb_ids, mutations = [], []
        for mutation in x.split('_'):
            pdb_ids.append(mutation.split(' ')[0])
            mutations.append(''.join(mutation.split(' ')[1:]))
        return '.'.join(pdb_ids), '.'.join(mutations)
    
    df = pd.read_csv(filename, names=column_names, comment='#')
    df['pdb_id'] = df['PDBFileID'].str.lower()
    df['pdb_chain'], df['pdb_mutation'] = zip(*df['Mutations'].apply(_parse_mutations))
    df['ddg_exp'] = df['DDG']
    # df.drop(['Mutations', 'DDG'], axis=1, inplace=True)
    df['uniprot_id'], df['uniprot_mutation'], df['pdb_chain_sifts'], df['pdb_mutation_sifts'] = zip(*(
        local.get_uniprot_id_mutation_rosetta_ddg(*x)
        for x in df[['pdb_id', 'pdb_chain', 'pdb_mutation']].values
    ))
    return df

## alascan_gpk

In [11]:
!grep '^#' $NOTEBOOK_NAME/ddg-master/input/csv/alascan-gpk.csv

# 
# This dataset of experimental alanine scanning data for single mutations was compiled by the Kortemme Lab, UCSF from the following curated datasets:
#  - Guerois et al. [1]
#  - Potapov et al. [2]
#  - Kellogg et al. [3]
# and the ProTherm database [4, 5].
# 
# The DSSPType values give the DSSP secondary structure type assigned by mkdssp 2.2.1 [6, 7]. The DSSPSimpleType values values
# give a simple elision of the DSSP types: G, H, and I become H(elix); E becomes S(heet); and T, B, S, and C become O(ther).
#     
# [1] PMID:12079393
# [2] PMID:19561092
# [3] PMID:21287615
# [4] PMID:9847203
# [5] PMID:16381846
# [6] PMID:21071423
# [7] PMID:6667333
# This dataset was last updated on 2015-01-29.
# The RecordID below refers to the record ID in the original dataset. When no ID was specified, we added an ID based on the published order of the records.
# Mutations is an underscore-separated list of mutations. Each mutation takes the form "Chain Wildtype ResidueID Mutant".
# DDG is the a

In [12]:
alascan_gpk = load_rosetta_ddg(op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/alascan-gpk.csv'))

PDB AA and UniProt AA are not the same! (1bf4, A, I, 17, None, 0) (I != T (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, I, 30, None, 0) (I != V (P13123))


Could not convert amino acid PTR to a one letter code!


In [13]:
display(alascan_gpk.head())
print(alascan_gpk.shape)

,RecordID,PDBFileID,Mutations,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,pdb_id,pdb_chain,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pdb_chain_sifts,pdb_mutation_sifts
0,69,1A2P,A Q 104 A,0.09,0.671958,S,O,0.16_0.02,1a2p,A,Q104A,0.09,P00648,Q151A,A,Q104A
1,68,1A2P,A I 109 A,1.63,0.214286,,O,1.27_2.22_1.4,1a2p,A,I109A,1.63,P00648,I156A,A,I109A
2,80,1A2P,A V 10 A,3.47,0.000000,H,H,3.47,1a2p,A,V10A,3.47,P00648,V57A,A,V10A
3,71,1A2P,A R 110 A,-0.07,0.315353,,O,-0.07,1a2p,A,R110A,-0.07,P00648,R157A,A,R110A
4,83,1A2P,A D 12 A,0.39,0.529801,H,H,0.39,1a2p,A,D12A,0.39,P00648,D59A,A,D12A


(768, 16)


In [14]:
df = alascan_gpk
df_name = 'alascan_gpk'

if not df['pdb_mutation'].str.contains('_').all():
    df['pdb_mutation'] = df['pdb_chain'] + '_' + df['pdb_mutation']
    
assert df['pdb_mutation'].str.contains('_').all()

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id', 'pdb_chain', 'pdb_mutation'), True],
        [('uniprot_id', 'uniprot_mutation'), False],
    ])

t.add_idx_column()

t.compress()

alascan_gpk


File size before: 0.07 MB
File size after: 0.04 MB
File size savings: 0.04 MB (53.08 %)


## curatedprotherm

In [15]:
!grep '^#' $NOTEBOOK_NAME/ddg-master/input/csv/curatedprotherm.csv

# 
# This dataset of single mutations was compiled by the Kortemme Lab, UCSF from the ProTherm database [1, 2].
# 
# The DSSPType column lists the DSSP secondary structure type assigned by mkdssp 2.2.1 [3, 4]. The DSSPSimpleSSType column
# lists a simple elision of the DSSP types: G, H, and I become H(elix); E becomes S(heet); and T, B, S, and C become O(ther).
#     
# [1] PMID:9847203
# [2] PMID:16381846
# [3] PMID:21071423
# [4] PMID:6667333
# This dataset was last updated on 2015-01-29.
# The RecordID below refers to the record ID in the original dataset. When no ID was specified, we added an ID based on the published order of the records.
# Mutations is an underscore-separated list of mutations. Each mutation takes the form "Chain Wildtype ResidueID Mutant".
# DDG is the aggregated (mean) DDG value used for analysis.
# IndividualDDGs lists the individual DDG values which can be used to filter out records with high variance.
# ResidueExposures is an underscore-separated list of exp

In [16]:
curatedprotherm = load_rosetta_ddg(op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/curatedprotherm.csv'))

Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid TRN to a one letter code!
Could not convert amino acid PTR to a one letter code!
Could not convert amino acid PTR to a one letter code!
Could not convert amino acid PTR to a one letter code!


No mutation mapping available! (1lve, I106L, A, I106L):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': nan, 'uniprot_id': nan, 'uniprot_aa': nan}]
Failed to get pdb_chain (A) and pdb_mutation (None) for input: (1lve, A, V27BL, None)
PDB AA and UniProt AA are not the same! (1lve, A, S, 29, None, 0) (S != N (P06312))
No mutation mapping available! (1lve, Y96P, A, Y96P):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': nan, 'uniprot_id': nan, 'uniprot_aa': nan}]
No mutation mapping available! (1lve, Y96Q, A, Y96Q):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': nan, 'uniprot_id': nan, 'uniprot_aa': nan}]
No mutation mapping available! (1lve, S97T, A, S97T):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': nan, 'uniprot_id': nan, 'uniprot_aa': nan}]
PDB AA and UniProt AA are not the same! (1myl, A, M, 31, None, 0) (M != R (P03050))
PDB AA and UniProt AA are not the same! (1myl, A, Y, 36, None, 0) (Y != E (P03050))
PDB AA and UniProt AA are not the same! (1myl, A

Could not convert amino acid PCA to a one letter code!
Could not convert amino acid CSO to a one letter code!


No mutation mapping available! (2imm, A100G, A, A100G):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': None, 'uniprot_id': None, 'uniprot_aa': None}]
No mutation mapping available! (2imm, L106I, A, L106I):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': None, 'uniprot_id': None, 'uniprot_aa': None}]
No mutation mapping available! (2imm, A15L, A, A15L):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': None, 'uniprot_id': None, 'uniprot_aa': None}]
No mutation mapping available! (2imm, M21I, A, M21I):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': None, 'uniprot_id': None, 'uniprot_aa': None}]
No mutation mapping available! (2imm, M21L, A, M21L):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': None, 'uniprot_id': None, 'uniprot_aa': None}]
No mutation mapping available! (2imm, K24R, A, K24R):
    [{'pdb_chain': 'A', 'pfam_id': nan, 'uniprot_pos': None, 'uniprot_id': None, 'uniprot_aa': None}]
No mutation mapping available! (2imm, F32Y, A, F32Y):
    [{

Could not convert amino acid ACE to a one letter code!
Could not convert amino acid ACE to a one letter code!
Could not convert amino acid ACE to a one letter code!


In [17]:
display(curatedprotherm.head())
print(curatedprotherm.shape)

,RecordID,PDBFileID,Mutations,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,pdb_id,pdb_chain,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pdb_chain_sifts,pdb_mutation_sifts
0,158,1A2P,A T 100 G,2.800000,0.506849,S,O,2.8,1a2p,A,T100G,2.800000,P00648,T147G,A,T100G
1,152,1A2P,A H 102 A,0.240000,0.510309,T,O,0.24,1a2p,A,H102A,0.240000,P00648,H149A,A,H102A
2,201,1A2P,A Y 103 F,0.090000,0.222707,T,O,0.18_0.0,1a2p,A,Y103F,0.090000,P00648,Y150F,A,Y103F
3,162,1A2P,A Q 104 A,0.090000,0.671958,S,O,0.16_0.02,1a2p,A,Q104A,0.090000,P00648,Q151A,A,Q104A
4,218,1A2P,A T 105 V,2.213333,0.575342,,O,2.14_2.25_2.25,1a2p,A,T105V,2.213333,P00648,T152V,A,T105V


(2971, 16)


In [18]:
df = curatedprotherm
df_name = 'curatedprotherm'

if not df['pdb_mutation'].str.contains('_').all():
    df['pdb_mutation'] = df['pdb_chain'] + '_' + df['pdb_mutation']
    
assert df['pdb_mutation'].str.contains('_').all()

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id', 'pdb_chain', 'pdb_mutation'), True],
        [('uniprot_id', 'uniprot_mutation'), False],
    ])

t.add_idx_column()

t.compress()

curatedprotherm


File size before: 0.30 MB
File size after: 0.14 MB
File size savings: 0.14 MB (47.58 %)


## guerois

In [19]:
!grep '^#' $NOTEBOOK_NAME/ddg-master/input/csv/guerois.csv

# 
# This dataset of mutations originates from the paper by Guerois et al. [1]. We have made some changes from the original
# dataset. In particular, we have tried to match the DDG values to the original publications.
# 
# The DSSPType column lists the DSSP secondary structure type assigned by mkdssp 2.2.1 [2, 3]. The DSSPSimpleSSType column
# lists a simple elision of the DSSP types: G, H, and I become H(elix); E becomes S(heet); and T, B, S, and C become O(ther).
#     
# [1] PMID:12079393
# [2] PMID:21071423
# [3] PMID:6667333
# This dataset was last updated on 2015-01-29.
# The RecordID below refers to the record ID in the original dataset. When no ID was specified, we added an ID based on the published order of the records.
# Mutations is an underscore-separated list of mutations. Each mutation takes the form "Chain Wildtype ResidueID Mutant".
# DDG is the aggregated (mean) DDG value used for analysis.
# IndividualDDGs lists the individual DDG values which can be used to filter ou

In [20]:
guerois = load_rosetta_ddg(op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/guerois.csv'))

PDB AA and UniProt AA are not the same! (107l, A, G, 44, None, 0) (G != S (P00720))
PDB AA and UniProt AA are not the same! (160l, A, A, 120, None, 0) (A != M (P00720))
PDB AA and UniProt AA are not the same! (161l, A, A, 116, None, 0) (A != N (P00720))
PDB AA and UniProt AA are not the same! (162l, A, A, 122, None, 0) (A != Q (P00720))
PDB AA and UniProt AA are not the same! (163l, A, A, 123, None, 0) (A != Q (P00720))
PDB AA and UniProt AA are not the same! (164l, A, A, 119, None, 0) (A != R (P00720))
PDB AA and UniProt AA are not the same! (165l, A, A, 117, None, 0) (A != S (P00720))
PDB AA and UniProt AA are not the same! (171l, A, A, 45, None, 0) (A != E (P00720))
PDB AA and UniProt AA are not the same! (1bf4, A, I, 17, None, 0) (I != T (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, I, 30, None, 0) (I != V (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, Q, 57, None, 0) (Q != D (P13123))
PDB AA and UniProt AA are not the same! (1dyb, A, G, 131, None, 0) (G 

In [21]:
display(guerois.head())
print(guerois.shape)

,RecordID,PDBFileID,Mutations,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,pdb_id,pdb_chain,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pdb_chain_sifts,pdb_mutation_sifts
0,965,107L,A G 44 S,-0.53,0.435294117647,H,H,0.53,107l,A,G44S,-0.53,NaN,NaN,A,G44S
1,966,160L,A A 120 M,-0.20,0.070796460177,H,H,0.20,160l,A,A120M,-0.20,NaN,NaN,A,A120M
2,967,161L,A A 116 N,0.17,0.212389380531,H,H,-0.17,161l,A,A116N,0.17,NaN,NaN,A,A116N
3,968,162L,A A 122 Q,-0.24,0.389380530973,H,H,0.24,162l,A,A122Q,-0.24,NaN,NaN,A,A122Q
4,969,163L,A A 123 Q,-0.22,0.46017699115,T,O,0.22,163l,A,A123Q,-0.22,NaN,NaN,A,A123Q


(1005, 16)


In [22]:
df = guerois
df_name = 'guerois'

if not df['pdb_mutation'].str.contains('_').all():
    df['pdb_mutation'] = df['pdb_chain'] + '_' + df['pdb_mutation']
    
assert df['pdb_mutation'].str.contains('_').all()

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id', 'pdb_chain', 'pdb_mutation'), True],
        [('uniprot_id', 'uniprot_mutation'), False],
    ])

t.add_idx_column()

t.compress()

guerois


File size before: 0.10 MB
File size after: 0.05 MB
File size savings: 0.05 MB (47.81 %)


## kellogg

In [23]:
!grep '^#' $NOTEBOOK_NAME/ddg-master/input/csv/kellogg.csv

# 
# This dataset of mutations originates from the paper by Kellogg et al. [1]. We have made some changes from the original
# dataset. In particular, we have tried to match the DDG values to the original publications.
# 
# The DSSPType column lists the DSSP secondary structure type assigned by mkdssp 2.2.1 [2, 3]. The DSSPSimpleSSType column
# lists a simple elision of the DSSP types: G, H, and I become H(elix); E becomes S(heet); and T, B, S, and C become O(ther).
#     
# [1] PMID:21287615
# [2] PMID:21071423
# [3] PMID:6667333
# This dataset was last updated on 2015-01-29.
# The RecordID below refers to the record ID in the original dataset. When no ID was specified, we added an ID based on the published order of the records.
# Mutations is an underscore-separated list of mutations. Each mutation takes the form "Chain Wildtype ResidueID Mutant".
# DDG is the aggregated (mean) DDG value used for analysis.
# IndividualDDGs lists the individual DDG values which can be used to filter ou

In [24]:
kellogg = load_rosetta_ddg(op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/kellogg.csv'))

PDB AA and UniProt AA are not the same! (107l, A, G, 44, None, 0) (G != S (P00720))
PDB AA and UniProt AA are not the same! (160l, A, A, 120, None, 0) (A != M (P00720))
PDB AA and UniProt AA are not the same! (161l, A, A, 116, None, 0) (A != N (P00720))
PDB AA and UniProt AA are not the same! (162l, A, A, 122, None, 0) (A != Q (P00720))
PDB AA and UniProt AA are not the same! (163l, A, A, 123, None, 0) (A != Q (P00720))
PDB AA and UniProt AA are not the same! (164l, A, A, 119, None, 0) (A != R (P00720))
PDB AA and UniProt AA are not the same! (165l, A, A, 117, None, 0) (A != S (P00720))
PDB AA and UniProt AA are not the same! (171l, A, A, 45, None, 0) (A != E (P00720))
PDB AA and UniProt AA are not the same! (1bf4, A, I, 17, None, 0) (I != T (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, I, 30, None, 0) (I != V (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, Q, 57, None, 0) (Q != D (P13123))
PDB AA and UniProt AA are not the same! (1dyb, A, G, 131, None, 0) (G 

In [25]:
display(kellogg.head())
print(kellogg.shape)

,RecordID,PDBFileID,Mutations,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,pdb_id,pdb_chain,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pdb_chain_sifts,pdb_mutation_sifts
0,1,107L,A G 44 S,-0.53,0.435294117647,H,H,0.53,107l,A,G44S,-0.53,NaN,NaN,A,G44S
1,2,160L,A A 120 M,-0.20,0.070796460177,H,H,0.2,160l,A,A120M,-0.20,NaN,NaN,A,A120M
2,3,161L,A A 116 N,0.17,0.212389380531,H,H,-0.17,161l,A,A116N,0.17,NaN,NaN,A,A116N
3,4,162L,A A 122 Q,-0.24,0.389380530973,H,H,0.24,162l,A,A122Q,-0.24,NaN,NaN,A,A122Q
4,5,163L,A A 123 Q,-0.22,0.46017699115,T,O,0.22,163l,A,A123Q,-0.22,NaN,NaN,A,A123Q


(1210, 16)


In [26]:
df = kellogg
df_name = 'kellogg'

if not df['pdb_mutation'].str.contains('_').all():
    df['pdb_mutation'] = df['pdb_chain'] + '_' + df['pdb_mutation']
    
assert df['pdb_mutation'].str.contains('_').all()

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id', 'pdb_chain', 'pdb_mutation'), True],
        [('uniprot_id', 'uniprot_mutation'), False],
    ])

t.add_idx_column()

t.compress()

File size before: 0.12 MB


kellogg


File size after: 0.06 MB
File size savings: 0.06 MB (46.00 %)


## potapov

In [27]:
!grep '^#' $NOTEBOOK_NAME/ddg-master/input/csv/potapov.csv

# 
# This dataset of mutations originates from the paper by Potapov et al. [1]. We have made some changes from the original
# dataset. In particular, we have tried to match the DDG values to the original publications.
# 
# The DSSPType column lists the DSSP secondary structure type assigned by mkdssp 2.2.1 [2, 3]. The DSSPSimpleSSType column
# lists a simple elision of the DSSP types: G, H, and I become H(elix); E becomes S(heet); and T, B, S, and C become O(ther).
#     
# [1] PMID:19561092
# [2] PMID:21071423
# [3] PMID:6667333
# This dataset was last updated on 2015-01-29.
# The RecordID below refers to the record ID in the original dataset. When no ID was specified, we added an ID based on the published order of the records.
# Mutations is an underscore-separated list of mutations. Each mutation takes the form "Chain Wildtype ResidueID Mutant".
# DDG is the aggregated (mean) DDG value used for analysis.
# IndividualDDGs lists the individual DDG values which can be used to filter ou

In [28]:
potapov = load_rosetta_ddg(op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/potapov.csv'))

PDB AA and UniProt AA are not the same! (1bf4, A, I, 17, None, 0) (I != T (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, I, 30, None, 0) (I != V (P13123))
PDB AA and UniProt AA are not the same! (1bf4, A, Q, 57, None, 0) (Q != D (P13123))


In [29]:
display(potapov.head())
print(potapov.shape)

,RecordID,PDBFileID,Mutations,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,pdb_id,pdb_chain,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pdb_chain_sifts,pdb_mutation_sifts
0,64,1A2P,A Y 103 F,0.00,0.222707423581,T,O,0.0,1a2p,A,Y103F,0.00,P00648,Y150F,A,Y103F
1,65,1A2P,A T 105 V,2.24,0.575342465753,,O,2.24,1a2p,A,T105V,2.24,P00648,T152V,A,T105V
2,66,1A2P,A I 109 A,2.07,0.214285714286,,O,2.07,1a2p,A,I109A,2.07,P00648,I156A,A,I109A
3,67,1A2P,A I 109 V,0.76,0.214285714286,,O,0.76,1a2p,A,I109V,0.76,P00648,I156V,A,I109V
4,9,1A2P,A V 10 A,3.39,0.0,H,H,3.39,1a2p,A,V10A,3.39,P00648,V57A,A,V10A


(2154, 16)


In [30]:
# An example of duplicates
potapov[
    (potapov['pdb_id'] == '1a2p') &
    (potapov['pdb_chain'] == 'A') &
    (potapov['pdb_mutation'] == 'A_Y103F')
]

,RecordID,PDBFileID,Mutations,DDG,ResidueExposures,DSSPTypes,DSSPSimpleTypes,IndividualDDGs,pdb_id,pdb_chain,pdb_mutation,ddg_exp,uniprot_id,uniprot_mutation,pdb_chain_sifts,pdb_mutation_sifts


In [31]:
print(potapov.shape[0])
print(potapov.drop_duplicates(subset=['pdb_id', 'pdb_chain', 'pdb_mutation']).shape[0])

2154
2144


In [32]:
potapov = potapov.drop_duplicates(subset=['pdb_id', 'pdb_chain', 'pdb_mutation'])

In [33]:
df = potapov
df_name = 'potapov'

if not df['pdb_mutation'].str.contains('_').all():
    df['pdb_mutation'] = df['pdb_chain'] + '_' + df['pdb_mutation']
    
assert df['pdb_mutation'].str.contains('_').all()

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id', 'pdb_chain', 'pdb_mutation'), True],
        [('uniprot_id', 'uniprot_mutation'), False],
    ])

t.add_idx_column()

t.compress()

potapov


File size before: 0.23 MB
File size after: 0.10 MB
File size savings: 0.10 MB (45.31 %)


# Load other info

## kortemme_baker

In [34]:
!head $NOTEBOOK_NAME/ddg-master/input/csv/kortemme_baker.csv -n 10

PDB_ID,Mutated_partner,Chain_ID,PDB_res#,Amino_acid,DDG_calc,DDG_obs,"Interface(O:no,1:yes)",PDBPosID,PubMedID,Location,FixedPDB_ID,FixedPDBResidueID
1A22,hGH,A,14,M,0,0.1,0,1A2214,7504735,Table_2,None,None
1A22,hGH,A,18,H,1.84,-0.5,1,1A2218,7504735,Table_2,None,None
1A22,hGH,A,21,H,0.78,0.2,1,1A2221,7504735,Table_2,None,None
1A22,hGH,A,22,Q,0.07,-0.2,1,1A2222,7504735,Table_2,None,None
1A22,hGH,A,25,F,1.3,-0.4,1,1A2225,7504735,Table_2,None,None
1A22,hGH,A,26,D,-0.05,-0.2,0,1A2226,7504735,Table_2,None,None
1A22,hGH,A,29,Q,0.01,-0.6,0,1A2229,7504735,Table_2,None,None
1A22,hGH,A,42,Y,2,0.2,1,1A2242,7504735,Table_2,None,None
1A22,hGH,A,45,L,1.15,1.2,1,1A2245,7504735,Table_2,None,None


In [35]:
kortemme_baker = (
    pd.read_csv(op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/kortemme_baker.csv'), header=0)
    .rename(columns={
        'PDB_ID': 'pdb_id',
        'Mutated_partner': 'mutated_partner',
        'Chain_ID': 'pdb_chain',
        'PDB_res#': 'pdb_resnum',
        'Amino_acid': 'pdb_aa_wt',
        'DDG_calc': 'ddg_calc',
        'DDG_obs': 'ddg_exp',
        'Interface(O:no,1:yes)': 'is_interface',
        'PDBPosID': 'pdb_pos_id',
        'PubMedID': 'pubmed_id',
        'Location': 'location',
        'FixedPDB_ID': 'fixed_pdb_id',
        'FixedPDBResidueID': 'fixed_pdb_residue_id',
    })
)

In [36]:
display(kortemme_baker.head())
print(kortemme_baker.shape)

,pdb_id,mutated_partner,pdb_chain,pdb_resnum,pdb_aa_wt,ddg_calc,ddg_exp,is_interface,pdb_pos_id,pubmed_id,location,fixed_pdb_id,fixed_pdb_residue_id
0,1A22,hGH,A,14,M,0.00,0.1,0,1A2214,7504735,Table_2,None,None
1,1A22,hGH,A,18,H,1.84,-0.5,1,1A2218,7504735,Table_2,None,None
2,1A22,hGH,A,21,H,0.78,0.2,1,1A2221,7504735,Table_2,None,None
3,1A22,hGH,A,22,Q,0.07,-0.2,1,1A2222,7504735,Table_2,None,None
4,1A22,hGH,A,25,F,1.30,-0.4,1,1A2225,7504735,Table_2,None,None


(380, 13)


In [37]:
db.import_tadf = potapov
df_name = 'potapov'

if not df['pdb_mutation'].str.contains('_').all():
    df['pdb_mutation'] = df['pdb_id'] + '_' + df['pdb_mutation']
    
assert df['pdb_mutation'].str.contains('_').all()

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id', 'pdb_chain', 'pdb_mutation'), True],
        [('uniprot_id', 'uniprot_mutation'), False],
    ])

t.add_idx_column()

t.compress()

potapov


File size before: 0.23 MB
File size after: 0.10 MB
File size savings: 0.10 MB (45.31 %)


## pdbs

In [38]:
!head -n 10 $NOTEBOOK_NAME/ddg-master/input/csv/pdbs.csv

#PDB ID,Resolution,Techniques
107L,1.8,X-RAY DIFFRACTION
160L,1.8,X-RAY DIFFRACTION
161L,1.7,X-RAY DIFFRACTION
162L,1.8,X-RAY DIFFRACTION
163L,1.8,X-RAY DIFFRACTION
164L,1.8,X-RAY DIFFRACTION
165L,1.75,X-RAY DIFFRACTION
171L,2.5,X-RAY DIFFRACTION
1A2P,1.5,X-RAY DIFFRACTION


In [39]:
pdbs = pd.read_csv(
    op.join(NOTEBOOK_NAME, 'ddg-master/input/csv/pdbs.csv'),
    names=['PDB ID', 'Resolution', 'Techniques'], 
    comment='#'
).rename(columns=lambda x: x.replace(' ', '_').lower())

In [40]:
display(pdbs.head())
print(pdbs.shape)

,pdb_id,resolution,techniques
0,107L,1.8,X-RAY DIFFRACTION
1,160L,1.8,X-RAY DIFFRACTION
2,161L,1.7,X-RAY DIFFRACTION
3,162L,1.8,X-RAY DIFFRACTION
4,163L,1.8,X-RAY DIFFRACTION


(182, 3)


In [41]:
pdbs['pdb_id'].str.lower().drop_duplicates().shape

(182,)

In [42]:
df = pdbs
df_name = 'pdbs'

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('pdb_id'), True],
    ])

t.add_idx_column()

t.compress()

File size before: 0.01 MB
File size after: 0.00 MB
File size savings: 0.00 MB (45.86 %)


pdbs


## references

In [43]:
!head -n 10 $NOTEBOOK_NAME/ddg-master/input/csv/references.csv

#ID,Authors,Title,Publication,Volume,Issue,Date,URL
PMID:10051585,Kay_Ramos_Baldwin,Specificity of native-like interhelical hydrophobic contacts in the apomyoglobin intermediate,Proceedings of the National Academy of Sciences,96,,1999-03-02,http://www.pnas.org/content/96/5/2007.abstract
PMID:10064710,Axe_Foster_Fersht,An irregular beta-bulge common to a group of bacterial RNases is an important determinant of stability and function in barnase,Journal of Molecular Biology,286,,1999-03-12,https://dx.doi.org/10.1006/jmbi.1999.2569
PMID:10064718,Peterson_Nicholson_Thapar_Klevit_Scholtz,Increased helix and protein stability through the introduction of a new tertiary hydrogen bond,Journal of Molecular Biology,286,,1999-03-12,https://dx.doi.org/10.1006/jmbi.1999.2574
PMID:10074361,Bishop_Ross_Kohanski,Autophosphorylation Dependent Destabilization of the Insulin Receptor Kinase Domain:? Tryptophan-1175 Reports Changes in the Catalytic Cleft†,Biochemistry,38,,1999-02-17,https://dx.doi.org/10.10

In [44]:
references = pd.read_csv(
    op.join(NOTEBOOK_NAME, 'ddg-master', 'input', 'csv', 'references.csv'),
    names=['ID', 'Authors', 'Title', 'Publication', 'Volume', 'Issue', 'Date', 'URL'], 
    comment='#'
).rename(columns=lambda x: x.lower())

In [45]:
display(references.head())
print(references.shape)

,id,authors,title,publication,volume,issue,date,url
0,PMID:10051585,Kay_Ramos_Baldwin,Specificity of native-like interhelical hydrop...,Proceedings of the National Academy of Sciences,96,NaN,1999-03-02,http://www.pnas.org/content/96/5/2007.abstract
1,PMID:10064710,Axe_Foster_Fersht,An irregular beta-bulge common to a group of b...,Journal of Molecular Biology,286,NaN,1999-03-12,https://dx.doi.org/10.1006/jmbi.1999.2569
2,PMID:10064718,Peterson_Nicholson_Thapar_Klevit_Scholtz,Increased helix and protein stability through ...,Journal of Molecular Biology,286,NaN,1999-03-12,https://dx.doi.org/10.1006/jmbi.1999.2574
3,PMID:10074361,Bishop_Ross_Kohanski,Autophosphorylation Dependent Destabilization ...,Biochemistry,38,NaN,1999-02-17,https://dx.doi.org/10.1021/bi982546s
4,PMID:10091653,Raffen_Dieckman_Szpunar_Wunschl_Pokkuluri_Dave...,Physicochemical consequences of amino acid var...,Protein Science,8,NaN,None,https://dx.doi.org/10.1110/ps.8.3.509


(454, 8)


In [46]:
df = references
df_name = 'citations'

t = db.import_df(df, df_name)

print(t.name)

t.create_indexes([
        [('id'), True],
    ])

t.add_idx_column()

t.compress()

citations


File size before: 0.11 MB
File size after: 0.06 MB
File size savings: 0.06 MB (58.02 %)


# Done

In [47]:
print(datetime.datetime.now())

2016-08-27 20:50:07.565238
